# Ekstraksi Informasi dari Berita Kriminal Indonesia menggunakan Named Entity Recognition (NER).

## Setting Up
### Install the required libraries

In [1]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords as nltk_stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory as SastrawiStopWordRemoverFactory
nltk.download('punkt')  

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Load Dataset

In [2]:
df = pd.read_csv("../data/processed/cleaned_all_data.csv", sep=',')
df.head()

,url,judul,tanggal,isi_berita,panjang_judul,panjang_isi_berita
0,https://www.detik.com/sumbagsel/hukum-dan-krim...,4 Anak di Bawah Umur Tersangka Pembunuhan di P...,2024-09-05 20:30:00,Pelaku pembunuhan dan pemerkosaan AA (14) seor...,73,2370
1,https://www.detik.com/jatim/hukum-dan-kriminal...,26 Tersangka Dibekuk Selama 3 Bulan Terakhir d...,2024-09-05 18:38:00,"Dalam waktu kurang lebih 3 bulan, Polres Probo...",64,1764
2,https://www.detik.com/sumut/hukum-dan-kriminal...,Pekerja Kafe Disiram Air Keras hingga Wajah 'B...,2024-09-04 21:40:00,"Seorang pekerja kafe di Cengkareng, MAS (32), ...",71,1995
3,https://www.detik.com/sumut/hukum-dan-kriminal...,Hilang Nyawa Pria di Simalungun gegara Rebutan...,2024-09-03 09:03:00,Hanya gegara rebutan mikrofon untuk bernyanyi ...,70,1398
4,https://www.detik.com/sumut/hukum-dan-kriminal...,Utang Rp 3 Juta Bikin Pegawai Akper Tewas di T...,2024-09-03 08:01:00,Hidup pria bernama Monika Hutauruk (45) harus ...,68,3907


## Preprocessing
### Tokenization

In [3]:
def remove_punctuation(text):
    if not isinstance(text, str):
        text = str(text)
    return re.sub(r'[^\w\s]', '', text)

df['judul_rp'] = df['judul'].apply(remove_punctuation)

df['isi_rp'] = df['isi_berita'].apply(remove_punctuation)

print("Hasil setelah membersihkan tanda baca secara terpisah:")
print(df[['judul', 'judul_rp', 'isi_berita', 'isi_rp']].head())

Hasil setelah membersihkan tanda baca secara terpisah:
                                               judul  \
0  4 Anak di Bawah Umur Tersangka Pembunuhan di P...   
1  26 Tersangka Dibekuk Selama 3 Bulan Terakhir d...   
2  Pekerja Kafe Disiram Air Keras hingga Wajah 'B...   
3  Hilang Nyawa Pria di Simalungun gegara Rebutan...   
4  Utang Rp 3 Juta Bikin Pegawai Akper Tewas di T...   

                                            judul_rp  \
0  4 Anak di Bawah Umur Tersangka Pembunuhan di P...   
1  26 Tersangka Dibekuk Selama 3 Bulan Terakhir d...   
2  Pekerja Kafe Disiram Air Keras hingga Wajah Be...   
3  Hilang Nyawa Pria di Simalungun gegara Rebutan...   
4  Utang Rp 3 Juta Bikin Pegawai Akper Tewas di T...   

                                          isi_berita  \
0  Pelaku pembunuhan dan pemerkosaan AA (14) seor...   
1  Dalam waktu kurang lebih 3 bulan, Polres Probo...   
2  Seorang pekerja kafe di Cengkareng, MAS (32), ...   
3  Hanya gegara rebutan mikrofon untuk bernyany

In [4]:
df['judul_rp'] = df['judul_rp'].fillna('')
df['isi_rp'] = df['isi_rp'].fillna('')

# Tokenisasi 
df['judul_tokens'] = df['judul_rp'].apply(word_tokenize)
df['isi_tokens'] = df['isi_rp'].apply(word_tokenize)

# Hasil tokenisasi
df[['judul_tokens', 'isi_tokens']].head()

,judul_tokens,isi_tokens
0,"[4, Anak, di, Bawah, Umur, Tersangka, Pembunuh...","[Pelaku, pembunuhan, dan, pemerkosaan, AA, 14,..."
1,"[26, Tersangka, Dibekuk, Selama, 3, Bulan, Ter...","[Dalam, waktu, kurang, lebih, 3, bulan, Polres..."
2,"[Pekerja, Kafe, Disiram, Air, Keras, hingga, W...","[Seorang, pekerja, kafe, di, Cengkareng, MAS, ..."
3,"[Hilang, Nyawa, Pria, di, Simalungun, gegara, ...","[Hanya, gegara, rebutan, mikrofon, untuk, bern..."
4,"[Utang, Rp, 3, Juta, Bikin, Pegawai, Akper, Te...","[Hidup, pria, bernama, Monika, Hutauruk, 45, h..."


### Casefolding


In [5]:
df['judul_tokens_lower'] = df['judul_tokens'].apply(lambda tokens: [token.lower() for token in tokens])
df['isi_tokens_lower'] = df['isi_tokens'].apply(lambda tokens: [token.lower() for token in tokens])

df[['judul_tokens_lower', 'isi_tokens_lower']].head()

,judul_tokens_lower,isi_tokens_lower
0,"[4, anak, di, bawah, umur, tersangka, pembunuh...","[pelaku, pembunuhan, dan, pemerkosaan, aa, 14,..."
1,"[26, tersangka, dibekuk, selama, 3, bulan, ter...","[dalam, waktu, kurang, lebih, 3, bulan, polres..."
2,"[pekerja, kafe, disiram, air, keras, hingga, w...","[seorang, pekerja, kafe, di, cengkareng, mas, ..."
3,"[hilang, nyawa, pria, di, simalungun, gegara, ...","[hanya, gegara, rebutan, mikrofon, untuk, bern..."
4,"[utang, rp, 3, juta, bikin, pegawai, akper, te...","[hidup, pria, bernama, monika, hutauruk, 45, h..."


### Stemming

In [6]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

df['judul_stemmed'] = df['judul_tokens_lower'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
df['isi_stemmed'] = df['isi_tokens_lower'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])

df[['judul_stemmed', 'isi_stemmed']].head()

,judul_stemmed,isi_stemmed
0,"[4, anak, di, bawah, umur, sangka, bunuh, di, ...","[laku, bunuh, dan, perkosa, aa, 14, orang, rem..."
1,"[26, sangka, bekuk, lama, 3, bulan, akhir, di,...","[dalam, waktu, kurang, lebih, 3, bulan, polres..."
2,"[kerja, kafe, siram, air, keras, hingga, wajah...","[orang, kerja, kafe, di, cengkareng, mas, 32, ..."
3,"[hilang, nyawa, pria, di, simalungun, gegara, ...","[hanya, gegara, rebut, mikrofon, untuk, nyanyi..."
4,"[utang, rp, 3, juta, bikin, pegawai, akper, te...","[hidup, pria, nama, monika, hutauruk, 45, haru..."


In [7]:
df.head()

,url,judul,tanggal,isi_berita,panjang_judul,panjang_isi_berita,judul_rp,isi_rp,judul_tokens,isi_tokens,judul_tokens_lower,isi_tokens_lower,judul_stemmed,isi_stemmed
0,https://www.detik.com/sumbagsel/hukum-dan-krim...,4 Anak di Bawah Umur Tersangka Pembunuhan di P...,2024-09-05 20:30:00,Pelaku pembunuhan dan pemerkosaan AA (14) seor...,73,2370,4 Anak di Bawah Umur Tersangka Pembunuhan di P...,Pelaku pembunuhan dan pemerkosaan AA 14 seoran...,"[4, Anak, di, Bawah, Umur, Tersangka, Pembunuh...","[Pelaku, pembunuhan, dan, pemerkosaan, AA, 14,...","[4, anak, di, bawah, umur, tersangka, pembunuh...","[pelaku, pembunuhan, dan, pemerkosaan, aa, 14,...","[4, anak, di, bawah, umur, sangka, bunuh, di, ...","[laku, bunuh, dan, perkosa, aa, 14, orang, rem..."
1,https://www.detik.com/jatim/hukum-dan-kriminal...,26 Tersangka Dibekuk Selama 3 Bulan Terakhir d...,2024-09-05 18:38:00,"Dalam waktu kurang lebih 3 bulan, Polres Probo...",64,1764,26 Tersangka Dibekuk Selama 3 Bulan Terakhir d...,Dalam waktu kurang lebih 3 bulan Polres Probol...,"[26, Tersangka, Dibekuk, Selama, 3, Bulan, Ter...","[Dalam, waktu, kurang, lebih, 3, bulan, Polres...","[26, tersangka, dibekuk, selama, 3, bulan, ter...","[dalam, waktu, kurang, lebih, 3, bulan, polres...","[26, sangka, bekuk, lama, 3, bulan, akhir, di,...","[dalam, waktu, kurang, lebih, 3, bulan, polres..."
2,https://www.detik.com/sumut/hukum-dan-kriminal...,Pekerja Kafe Disiram Air Keras hingga Wajah 'B...,2024-09-04 21:40:00,"Seorang pekerja kafe di Cengkareng, MAS (32), ...",71,1995,Pekerja Kafe Disiram Air Keras hingga Wajah Be...,Seorang pekerja kafe di Cengkareng MAS 32 didu...,"[Pekerja, Kafe, Disiram, Air, Keras, hingga, W...","[Seorang, pekerja, kafe, di, Cengkareng, MAS, ...","[pekerja, kafe, disiram, air, keras, hingga, w...","[seorang, pekerja, kafe, di, cengkareng, mas, ...","[kerja, kafe, siram, air, keras, hingga, wajah...","[orang, kerja, kafe, di, cengkareng, mas, 32, ..."
3,https://www.detik.com/sumut/hukum-dan-kriminal...,Hilang Nyawa Pria di Simalungun gegara Rebutan...,2024-09-03 09:03:00,Hanya gegara rebutan mikrofon untuk bernyanyi ...,70,1398,Hilang Nyawa Pria di Simalungun gegara Rebutan...,Hanya gegara rebutan mikrofon untuk bernyanyi ...,"[Hilang, Nyawa, Pria, di, Simalungun, gegara, ...","[Hanya, gegara, rebutan, mikrofon, untuk, bern...","[hilang, nyawa, pria, di, simalungun, gegara, ...","[hanya, gegara, rebutan, mikrofon, untuk, bern...","[hilang, nyawa, pria, di, simalungun, gegara, ...","[hanya, gegara, rebut, mikrofon, untuk, nyanyi..."
4,https://www.detik.com/sumut/hukum-dan-kriminal...,Utang Rp 3 Juta Bikin Pegawai Akper Tewas di T...,2024-09-03 08:01:00,Hidup pria bernama Monika Hutauruk (45) harus ...,68,3907,Utang Rp 3 Juta Bikin Pegawai Akper Tewas di T...,Hidup pria bernama Monika Hutauruk 45 harus be...,"[Utang, Rp, 3, Juta, Bikin, Pegawai, Akper, Te...","[Hidup, pria, bernama, Monika, Hutauruk, 45, h...","[utang, rp, 3, juta, bikin, pegawai, akper, te...","[hidup, pria, bernama, monika, hutauruk, 45, h...","[utang, rp, 3, juta, bikin, pegawai, akper, te...","[hidup, pria, nama, monika, hutauruk, 45, haru..."


In [ ]:
df.to_csv("../data/processed/stemmed.csv", sep=',', index=False)

### Stop Word Removal & POS Tagging

In [15]:
df = pd.read_csv("../data/processed/stemmed.csv", sep=',')
df.head()

,url,judul,tanggal,isi_berita,panjang_judul,panjang_isi_berita,judul_rp,isi_rp,judul_tokens,isi_tokens,judul_tokens_lower,isi_tokens_lower,judul_stemmed,isi_stemmed
0,https://www.detik.com/sumbagsel/hukum-dan-krim...,4 Anak di Bawah Umur Tersangka Pembunuhan di P...,2024-09-05 20:30:00,Pelaku pembunuhan dan pemerkosaan AA (14) seor...,73,2370,4 Anak di Bawah Umur Tersangka Pembunuhan di P...,Pelaku pembunuhan dan pemerkosaan AA 14 seoran...,"['4', 'Anak', 'di', 'Bawah', 'Umur', 'Tersangk...","['Pelaku', 'pembunuhan', 'dan', 'pemerkosaan',...","['4', 'anak', 'di', 'bawah', 'umur', 'tersangk...","['pelaku', 'pembunuhan', 'dan', 'pemerkosaan',...","['4', 'anak', 'di', 'bawah', 'umur', 'sangka',...","['laku', 'bunuh', 'dan', 'perkosa', 'aa', '14'..."
1,https://www.detik.com/jatim/hukum-dan-kriminal...,26 Tersangka Dibekuk Selama 3 Bulan Terakhir d...,2024-09-05 18:38:00,"Dalam waktu kurang lebih 3 bulan, Polres Probo...",64,1764,26 Tersangka Dibekuk Selama 3 Bulan Terakhir d...,Dalam waktu kurang lebih 3 bulan Polres Probol...,"['26', 'Tersangka', 'Dibekuk', 'Selama', '3', ...","['Dalam', 'waktu', 'kurang', 'lebih', '3', 'bu...","['26', 'tersangka', 'dibekuk', 'selama', '3', ...","['dalam', 'waktu', 'kurang', 'lebih', '3', 'bu...","['26', 'sangka', 'bekuk', 'lama', '3', 'bulan'...","['dalam', 'waktu', 'kurang', 'lebih', '3', 'bu..."
2,https://www.detik.com/sumut/hukum-dan-kriminal...,Pekerja Kafe Disiram Air Keras hingga Wajah 'B...,2024-09-04 21:40:00,"Seorang pekerja kafe di Cengkareng, MAS (32), ...",71,1995,Pekerja Kafe Disiram Air Keras hingga Wajah Be...,Seorang pekerja kafe di Cengkareng MAS 32 didu...,"['Pekerja', 'Kafe', 'Disiram', 'Air', 'Keras',...","['Seorang', 'pekerja', 'kafe', 'di', 'Cengkare...","['pekerja', 'kafe', 'disiram', 'air', 'keras',...","['seorang', 'pekerja', 'kafe', 'di', 'cengkare...","['kerja', 'kafe', 'siram', 'air', 'keras', 'hi...","['orang', 'kerja', 'kafe', 'di', 'cengkareng',..."
3,https://www.detik.com/sumut/hukum-dan-kriminal...,Hilang Nyawa Pria di Simalungun gegara Rebutan...,2024-09-03 09:03:00,Hanya gegara rebutan mikrofon untuk bernyanyi ...,70,1398,Hilang Nyawa Pria di Simalungun gegara Rebutan...,Hanya gegara rebutan mikrofon untuk bernyanyi ...,"['Hilang', 'Nyawa', 'Pria', 'di', 'Simalungun'...","['Hanya', 'gegara', 'rebutan', 'mikrofon', 'un...","['hilang', 'nyawa', 'pria', 'di', 'simalungun'...","['hanya', 'gegara', 'rebutan', 'mikrofon', 'un...","['hilang', 'nyawa', 'pria', 'di', 'simalungun'...","['hanya', 'gegara', 'rebut', 'mikrofon', 'untu..."
4,https://www.detik.com/sumut/hukum-dan-kriminal...,Utang Rp 3 Juta Bikin Pegawai Akper Tewas di T...,2024-09-03 08:01:00,Hidup pria bernama Monika Hutauruk (45) harus ...,68,3907,Utang Rp 3 Juta Bikin Pegawai Akper Tewas di T...,Hidup pria bernama Monika Hutauruk 45 harus be...,"['Utang', 'Rp', '3', 'Juta', 'Bikin', 'Pegawai...","['Hidup', 'pria', 'bernama', 'Monika', 'Hutaur...","['utang', 'rp', '3', 'juta', 'bikin', 'pegawai...","['hidup', 'pria', 'bernama', 'monika', 'hutaur...","['utang', 'rp', '3', 'juta', 'bikin', 'pegawai...","['hidup', 'pria', 'nama', 'monika', 'hutauruk'..."


In [11]:
pip install flair

  Using cached flair-0.15.1-py3-none-any.whl.metadata (12 kB)
  Using cached boto3-1.38.23-py3-none-any.whl.metadata (6.6 kB)
  Using cached conllu-4.5.3-py2.py3-none-any.whl.metadata (19 kB)
  Using cached Deprecated-1.2.18-py2.py3-none-any.whl.metadata (5.7 kB)
  Using cached ftfy-6.3.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached gdown-5.2.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached langdetect-1.0.9.tar.gz (981 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached lxml-5.4.0-cp311-cp311-win_amd64.whl.metadata (3.6 kB)
  Using cached more_itertools-10.7.0-py3-none-any.whl.metadata (37 kB)
  Using cached mpld3-0.5.10-py3-none-any.whl.metadata (5.1 kB)
  Using cached pptree-3.1.tar.gz (3.0 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pytorch_revgrad-0.2.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached segtok-1.5.11-py3-none-any.w

  DEPRECATION: Building 'langdetect' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'langdetect'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  DEPRECATION: Building 'pptree' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'pptree'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  DEPRECATION: Building 'sqlitedict' using the legacy setup.py bdist_wheel mecha

In [ ]:
import pandas as pd
import ast # Untuk mengubah string representasi list menjadi list aktual

# Pastikan Pustaka Sastrawi telah terinstal jika Anda ingin menggunakannya secara penuh
# Jika tidak, placeholder stopwords akan digunakan.
try:
    from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
    stopword_factory = StopWordRemoverFactory()
    stopwords_set = set(stopword_factory.get_stop_words())
    print("Berhasil memuat stopwords dari Sastrawi.")
except ImportError:
    print("Pustaka Sastrawi tidak ditemukan. Menggunakan placeholder stopwords.")
    print("Anda dapat menginstal Sastrawi dengan: pip install Sastrawi")
    stopwords_set = set(['yang', 'untuk', 'pada', 'ke', 'para', 'namun', 'menurut', 'antara', 'dia', 'dua', 'ia', 'seperti', 'jika', 'jika', 'maka', 'sampai', 'saat', 'hal', 'akan', 'lagi', 'telah', 'oleh', 'sebagai', 'dan', 'di', 'dari']) # Contoh stopwords

# --- Pengaturan Flair POS Tagger ---
# Pastikan Anda telah menginstal flair: pip install flair
try:
    from flair.data import Sentence
    from flair.models import SequenceTagger
    tagger = SequenceTagger.load('pos-multi')
    print("Model Flair POS tagger berhasil dimuat.")
except ImportError:
    print("Pustaka Flair tidak ditemukan. Silakan install dengan: pip install flair")
    tagger = None
except Exception as e:
    print(f"Error saat memuat model Flair POS tagger: {e}")
    print("Pastikan Anda memiliki koneksi internet untuk pengunduhan pertama.")
    tagger = None # Atur tagger ke None jika gagal memuat

# --- Fungsi-fungsi Pemrosesan ---

def safe_literal_eval(val):
    """Mengubah string representasi list menjadi list, menangani error dan NaN."""
    if pd.isna(val):
        return []
    try:
        return ast.literal_eval(val)
    except (ValueError, SyntaxError):
        # Jika bukan string list yang valid, kembalikan list kosong atau tangani sesuai kebutuhan
        # Misalnya, jika itu adalah string biasa yang tidak dimaksudkan sebagai list, Anda mungkin ingin men-splitnya.
        # Untuk saat ini, kita asumsikan input adalah stringified list atau NaN.
        print(f"Peringatan: Gagal mengurai string sebagai list: {val}. Mengembalikan list kosong.")
        return []


def remove_stopwords(tokens):
    """Menghapus stopwords dari daftar token."""
    if not isinstance(tokens, list):
        # Ini mungkin terjadi jika safe_literal_eval mengembalikan sesuatu selain list
        # atau jika kolom asli tidak berisi stringified list.
        print(f"Peringatan: Input untuk remove_stopwords bukan list: {tokens}. Mengembalikan list kosong.")
        return []
    return [token for token in tokens if token.lower() not in stopwords_set and token.strip() != '']

def pos_tag_tokens_with_flair(tokens):
    """
    Melakukan POS tagging pada daftar token menggunakan Flair.
    Input: Daftar string (token).
    Output: Daftar tuple (token, pos_tag).
    """
    if not tagger:
        print("Tagger Flair tidak dimuat. Mengembalikan POS tags kosong.")
        return [(token, 'UNKN') for token in tokens]
    
    if not tokens or not isinstance(tokens, list) or not all(isinstance(t, str) for t in tokens):
        # Jika tokens adalah list kosong setelah stopword removal, itu valid.
        if isinstance(tokens, list) and not tokens:
            return []
        print(f"Peringatan: Input untuk pos_tag_tokens_with_flair tidak valid: {tokens}. Mengembalikan list kosong.")
        return []

    sentence_text = " ".join(tokens)
    sentence = Sentence(sentence_text)

    try:
        tagger.predict(sentence)
    except Exception as e:
        print(f"Error saat prediksi Flair: {e}")
        return [(token, 'ERR') for token in tokens]

    pos_tags_from_flair = []
    for token_obj in sentence.tokens:
        pos_tags_from_flair.append((token_obj.text, token_obj.get_tag('pos').value))
    
    # Penanganan ketidakcocokan tokenisasi antara input dan Flair
    if len(pos_tags_from_flair) == len(tokens):
        return pos_tags_from_flair
    else:
        print(f"Peringatan: Jumlah token tidak cocok setelah Flair. Input: {len(tokens)} token, Flair: {len(pos_tags_from_flair)} token.")
        print(f"Token input: {tokens}")
        print(f"Tag Flair: {[(t.text, t.get_tag('pos').value) for t in sentence.tokens]}")
        
        # Implementasi penyelarasan (alignment)
        realigned_tags = []
        flair_token_texts = [pt[0] for pt in pos_tags_from_flair]
        
        original_idx = 0
        flair_idx = 0

        while original_idx < len(tokens):
            original_token = tokens[original_idx]
            current_reconstructed_flair_segment = ""
            tags_for_current_original_token = []
            
            temp_flair_idx_start = flair_idx
            while flair_idx < len(flair_token_texts):
                segment_to_add = flair_token_texts[flair_idx]
                
                # Normalisasi untuk perbandingan (misalnya, hilangkan spasi jika Flair memecah token secara berbeda)
                # Ini adalah logika yang disederhanakan; kasus kompleks mungkin memerlukan penanganan lebih lanjut.
                combined_segment = (current_reconstructed_flair_segment + segment_to_add).replace(" ", "")
                original_token_compare = original_token.replace(" ", "")

                if combined_segment == original_token_compare:
                    current_reconstructed_flair_segment += segment_to_add
                    tags_for_current_original_token.append(pos_tags_from_flair[flair_idx][1])
                    flair_idx += 1
                    break 
                elif original_token_compare.startswith(combined_segment):
                    current_reconstructed_flair_segment += segment_to_add
                    tags_for_current_original_token.append(pos_tags_from_flair[flair_idx][1])
                    flair_idx += 1
                else:
                    break 
            
            if current_reconstructed_flair_segment.replace(" ", "") == original_token.replace(" ", "") and tags_for_current_original_token:
                realigned_tags.append((original_token, tags_for_current_original_token[0])) # Ambil tag pertama
            else:
                realigned_tags.append((original_token, 'UNKN_ALIGN')) # Tandai jika penyelarasan gagal
                flair_idx = temp_flair_idx_start # Reset flair_idx untuk coba lagi dengan token asli berikutnya
                # Untuk menghindari infinite loop jika ada ketidakcocokan persisten:
                if flair_idx < len(flair_token_texts) and original_idx +1 < len(tokens) :
                     # Coba majukan flair_idx jika token berikutnya tidak cocok, ini spekulatif
                    if not tokens[original_idx+1].startswith(flair_token_texts[flair_idx]):
                         flair_idx +=1
                elif flair_idx == len(flair_token_texts) and original_idx +1 < len(tokens):
                    # Flair tokens habis, tapi masih ada original tokens
                    pass # Akan ditandai UNKN_ALIGN di iterasi berikutnya

            original_idx += 1
            
        if len(realigned_tags) == len(tokens):
            return realigned_tags
        else: # Fallback jika panjang akhir tidak sama
            print("Penyelarasan akhir gagal menghasilkan jumlah token yang sama. Mengembalikan token asli dengan tag UNKN.")
            return [(token, 'UNKN_FINAL') for token in tokens]

file_path = 'cleaned_all_data.csv' 

judul_stemmed_input_col = 'judul_stemmed'
isi_stemmed_input_col = 'isi_stemmed'

original_judul_display_col = 'judul'
original_isi_display_col = 'isi_berita' 

try:
    df = pd.read_csv(file_path)
    print(f"\nBerhasil memuat DataFrame dari {file_path}.")
    print("Kolom yang tersedia:", df.columns.tolist())

    missing_cols = []
    if judul_stemmed_input_col not in df.columns:
        missing_cols.append(judul_stemmed_input_col)
    if isi_stemmed_input_col not in df.columns:
        missing_cols.append(isi_stemmed_input_col)
    
    if missing_cols:
        print(f"Error: Kolom input berikut tidak ditemukan di CSV: {', '.join(missing_cols)}. Harap periksa file CSV dan nama kolom.")
    elif not tagger:
        print("\nPOS tagging dengan Flair tidak dapat dilakukan karena tagger gagal dimuat.")
    else:
        print(f"\nMemproses kolom '{judul_stemmed_input_col}' dan '{isi_stemmed_input_col}'...")

        # 1. Konversi stringified list menjadi list aktual
        df['judul_stemmed_list'] = df[judul_stemmed_input_col].apply(safe_literal_eval).copy()
        df['isi_stemmed_list'] = df[isi_stemmed_input_col].apply(safe_literal_eval).copy()
        print("Langkah 1: Konversi kolom stemmed menjadi list selesai.")

        # 2. Penghapusan Stopword
        df['judul_stemmed_nostop'] = df['judul_stemmed_list'].apply(remove_stopwords)
        df['isi_stemmed_nostop'] = df['isi_stemmed_list'].apply(remove_stopwords)
        print("Langkah 2: Penghapusan Stopword selesai.")

        # 3. POS Tagging dengan Flair
        df['judul_pos_flair'] = df['judul_stemmed_nostop'].apply(pos_tag_tokens_with_flair)
        df['isi_pos_flair'] = df['isi_stemmed_nostop'].apply(pos_tag_tokens_with_flair)
        print("Langkah 3: POS Tagging dengan Flair selesai.")
        
        print("\nContoh hasil pemrosesan DataFrame:")
        
        display_cols_judul = []
        if original_judul_display_col in df.columns:
            display_cols_judul.append(original_judul_display_col)
        display_cols_judul.extend([judul_stemmed_input_col, 'judul_stemmed_nostop', 'judul_pos_flair'])
        
        display_cols_isi = []
        if original_isi_display_col in df.columns:
            display_cols_isi.append(original_isi_display_col)
        display_cols_isi.extend([isi_stemmed_input_col, 'isi_stemmed_nostop', 'isi_pos_flair'])

        print("\n--- Judul ---")
        print(df[display_cols_judul].head())
        print("\n--- Isi ---")
        print(df[display_cols_isi].head())

        if not df['isi_pos_flair'].empty and df['isi_pos_flair'].iloc[0] is not None and len(df['isi_pos_flair'].iloc[0]) > 0 :
            print("\nContoh detail 'isi_pos_flair' untuk entri pertama:")
            for token, tag in df['isi_pos_flair'].iloc[0]:
                print(f"Token: {token}, POS: {tag}")
        else:
            print(f"\nTidak ada data 'isi_pos_flair' untuk ditampilkan pada entri pertama atau kolom '{isi_stemmed_input_col}' mungkin kosong setelah diproses.")

except FileNotFoundError:
    print(f"Error: File '{file_path}' tidak ditemukan. Pastikan file ada di direktori yang benar dan nama file sudah sesuai.")
except pd.errors.EmptyDataError:
    print(f"Error: File '{file_path}' kosong.")
except Exception as e:
    print(f"Terjadi error saat memproses DataFrame: {e}")
    import traceback
    print(traceback.format_exc())

Berhasil memuat stopwords dari Sastrawi.


c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.flair\models\upos-multi\models--flair--upos-multi. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administ

2025-05-24 11:58:21,777 SequenceTagger predicts: Dictionary with 17 tags: NOUN, PUNCT, ADP, VERB, ADJ, DET, PROPN, ADV, PRON, AUX, CCONJ, NUM, SCONJ, PART, X, SYM, INTJ
Model Flair POS tagger berhasil dimuat.
Error: File 'cleaned_all_data.csv' tidak ditemukan. Pastikan file ada di direktori yang benar.


In [ ]:
df[['judul', 'judul_stemmed_nostop', 'judul_pos_nostop', 'isi_stemmed_nostop', 'isi_pos_nostop']].head()

In [ ]:
df.to_csv("../data/processed/prepocessed_all_data.csv", sep=',', index=False)